In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager # sustituye al archivo
from selenium.webdriver.chrome.options import Options # opciones de chrome
from selenium.webdriver.common.by import By # By es para buscar por tag, clase, id...
from selenium.webdriver.support.ui import WebDriverWait   # para meter esperaras
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver import ActionChains # para hacer acciones con el ratón
from selenium.webdriver.support import expected_conditions as EC   # para esperar ciertos eventos
from selenium.webdriver.chrome.service import Service
import selenium.webdriver.common.keys as Keys # para simular teclas
import pandas as pd

In [ ]:
#Opciones de chrome
opciones = webdriver.ChromeOptions()
#opciones.add_argument('--start-maximized')
#opciones.add_argument("--headless")
opciones.add_argument('window-size=775,1400')
opciones.add_argument('--disable-extensions')
opciones.add_argument('--disable-blink-features=AutomationControlled')
opciones.add_argument('--no-sandbox')
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

# guardar las cookies
# opciones.add_argument('user-data-dir=cookies')    # mantiene las cookies
opciones.add_argument('--incognito') # modo incognito

In [ ]:
url = "https://kadarceramica.com/"
driver = webdriver.Chrome(options=opciones)
driver.get(url)

In [ ]:
#directamente de la página principal extraigo las categorias de los productos
lista = driver.find_element(By.CLASS_NAME, "product-categories")
categorias = lista.find_elements(By.TAG_NAME, "li")

In [ ]:
# con este bucle limpio los elementos que he cogido, y me quedo solo con los enlaces de 
# categorías que no tienen subcategorias, o de las subcategorías
# son links que me llevan directamente a páginas que contienen productos
lista_link_categorias = []
for e in categorias:
    cat = e.get_attribute("class")
    if "cat-parent" in cat:
        print("esta no")
    else:
        lista_link_categorias.append(e)

In [ ]:
#ahora saco los href con los que voy a trabajar
lista_categorias = []
for e in lista_link_categorias:
    enlace = e.find_element(By.TAG_NAME, "a")
    lista_categorias.append(enlace.get_attribute("href"))

In [ ]:
# Inicializar el driver
driver2 = webdriver.Chrome(options=opciones)
driver3 = webdriver.Chrome(options=opciones)
lista_productos = []
i = 1

# Recorrer las categorías y almacenar los enlaces de los productos
for url in lista_categorias:
    driver2.get(url)

    while True:
        productos = driver2.find_elements(By.CLASS_NAME, "woocommerce-loop-product__link")
        for pro in productos:
            lista_productos.append(pro.get_attribute("href"))
        try:
            driver2.find_element(By.CLASS_NAME, "next").click()
            driver2.implicitly_wait(5)
        except: 
            print(f"página {i}, {url}")
            i+=1
            break
driver2.quit()

# Almacenar la información de los productos
productos_info = []

#recorro la lista con los productos, accediendo a cada uno de ellos y almacenando la información que necesito
for producto_url in lista_productos:
    driver3.get(producto_url)
    try:
        cat = driver3.find_element(By.CLASS_NAME, "woocommerce-breadcrumb")
        lista_separada = cat.text.split(" / ")
        if len(lista_separada) == 4:
            categoria = lista_separada[1]
            subcategoria = lista_separada[2]
            nombre = lista_separada[3]
        else: 
            categoria = lista_separada[1]
            subcategoria = "no tiene"
            nombre = lista_separada[2]
        # Intentar encontrar el precio en diferentes estructuras HTML
        try:
            precio_elemento = WebDriverWait(driver3, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "p.price span.woocommerce-Price-amount bdi"))
            )
        except:
            try:
                precio_elemento = WebDriverWait(driver3, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span.price span.woocommerce-Price-amount bdi"))
                )
            except:
                precio_elemento = WebDriverWait(driver3, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "span.woocommerce-Price-amount bdi"))
                )
        
        precio = precio_elemento.text
        try: 
            descripcion = driver3.find_element(By.CLASS_NAME, "woocommerce-Tabs-panel--description").text
        except:
            descripcion = "no tiene"
        
        # Crear el diccionario con la información del producto
        producto_dict = {
            "categoria": categoria,
            "subcategoria": subcategoria,
            "nombre": nombre,
            "precio": precio,
            "descripcion": descripcion,
            "url": producto_url,
            "Tienda": "Kadar"  # Agregar la tienda
        }
        
        # Agregar el diccionario a la lista de productos_info
        productos_info.append(producto_dict)
    except Exception as e:
        print(f"Error al procesar {producto_url}: {e}. Error {i}")
        i+=1

driver3.quit()

# Crear un DataFrame con la lista de productos_info
df_productos = pd.DataFrame(productos_info)

# Guardar el DataFrame en un archivo CSV
df_productos.to_csv("kadar.csv", index=False)

# Imprimir el mensaje de confirmación
print("Datos guardados correctamente en kadar.csv")
